# Projet scoring

## Contexte du projet

Système expert Best Immo est un outil d'aide à la décision à l'octroi d'un crédit immobilier. Il permet de rendre un avis en risque pour un projet immobilier à partir des données saisies par le conseiller. 
Cette note de risque résulte d'un croisement entre un diagnostic de solvabilité qui a été revu fin 2019 par la direction des engagements et d'un score statistique construit et maintenu par RCP/Modélisation. 
Le score statistique a été mise en production en 2014 et 2015. La variable cible d score est le défaut actuel observé à 36 mois suite à l'octroi. 





  Seuil d’arriérés franchis + de 90 jours consécutifs (seuil absolu : supérieurs à 100 € Retail / à 500 € Corporate et seuil relatif : supérieurs à 1% du bilan) ▪ Faillite / procédure judiciaire (surendettement Banque de France, RJ / LJ, procédure ou plan de sauvegarde) ▪ Les restructurations pour risque avec perte économique > 1% (baisse du taux d’intérêt) ▪ Contagion ▪ Perspectives négatives (avis d’expert basé sur des indicateurs d’alertes précoces - Early Warnings) ▪ Client en défaut chez Interfimo

## Objectifs


Le but est de:
- prendre en compte un profil emprunteur actualisé avec l'utilisation d'un historique de données plus récent. 
- d'explorer et étudier l'apport de nouvelles variables discriminantes
- tester l'apport de données externes (données socio-démo par ex)
- prendre en compte la nouvelle définition du défaut balois comme variable cible
(meilleure quantification du risque de défaut et évolution réglementaire)



## Exploration des données 

In [1]:
import pandas as pd
import os

In [16]:
data_path=os.getcwd()+"/projet/data"

In [18]:
data_path


'/Users/luciegabagnou/Documents/MOSEF/SCORING/projet/data'

In [19]:
raw_data=pd.read_excel(f"{data_path}/Bases_recents.xlsx")

In [9]:
raw_data

,region_cri,date_debloc_avec_crd,DAT_VALIDE_ACCORD_CRI,nb_pret,TOP_CONNU_BRP,TOP_SCI_BRP,ASU_BIEN_FIN_BRP,NAT_BIEN_FIN_BRP,NBR_INT_BRP,NBR_TOT_COEMPR_CRI,...,Mt_credit_nb_ans_rev_prof,Epargne_nb_ans_rev_prof,rentabilite_loc,financement_tot,quotite,top_locatif,top_pret_int_ext,top_autre_pret_int,top_autre_pret_ext,top_pers_seule
0,ILE DE FRANCE,201405,2010-12-01,1,0,0,40,10,2,1,...,3.140729,NaN,NaN,301510.0,1.083166,1,0,0,0,1
1,ILE DE FRANCE,201405,2011-03-14,2,0,0,10,10,2,1,...,0.999205,NaN,NaN,190962.0,0.310507,0,0,0,0,0
2,PACA,201407,2011-06-21,1,0,0,40,20,1,0,...,4.375182,NaN,NaN,105000.0,1.000000,1,0,0,0,1
3,AUVERGNE,201404,2011-06-27,1,0,0,20,20,2,1,...,2.006637,0.389825,NaN,215280.0,0.800000,0,0,0,0,0
4,ILE DE FRANCE,201404,2011-09-29,3,0,0,10,10,1,0,...,4.536824,0.287687,NaN,177700.0,0.585158,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114411,PAYS DE LA LOIRE,201804,2018-04-06,1,0,0,10,20,2,1,...,3.743370,2.582877,NaN,252670.0,1.052792,0,0,0,0,1
114412,AUVERGNE,201804,2018-03-29,2,0,0,10,10,1,0,...,3.726499,NaN,NaN,244000.0,1.000000,0,0,0,0,1
114413,ILE DE FRANCE,201804,2018-03-28,1,0,0,10,10,2,1,...,6.028812,0.697618,NaN,345680.0,1.002698,0,0,0,0,1
114414,BRETAGNE,201804,2018-03-29,1,0,0,20,20,2,1,...,3.269921,NaN,NaN,147330.0,1.091333,0,0,0,0,0


In [11]:
raw_data.columns.tolist()

['region_cri',
 'date_debloc_avec_crd',
 'DAT_VALIDE_ACCORD_CRI',
 'nb_pret',
 'TOP_CONNU_BRP',
 'TOP_SCI_BRP',
 'ASU_BIEN_FIN_BRP',
 'NAT_BIEN_FIN_BRP',
 'NBR_INT_BRP',
 'NBR_TOT_COEMPR_CRI',
 'COD_POSTAL_BIEN_CRI',
 'TOP_GARANTIE_CL',
 'CODPAY_BIEN_CRI',
 'CODTYPE_PROJET_CRI',
 'COD_CPPOP_CRI',
 'COD_ETA_BIEN_CRI',
 'COD_USAGE_BIEN_CRI',
 'BEST_APPORT_MT_BRP',
 'BEST_APPORT_TX_BRP',
 'PCT_APPORT_PERS_CRI',
 'PCT_ENDETTMT_CRI',
 'TOP_ETR_BRP',
 'TOP_PRET_RELAIS_BRP',
 'TX_APPORT_HF_AT_BRP',
 'TX_APPORT_HF_HT_BRP',
 'TX_APPORT_TF_AT_BRP',
 'TX_APPORT_TF_HT_BRP',
 'TX_APPORT_BRP',
 'TX_APPORT_AGENCE_BRP',
 'TX_FINANCEMENT_BRP',
 'TX_FINANCEMENT_AGENCE_BRP',
 'FINANCEMENT_LT_BRP',
 'FINANCEMENT_PR_BRP',
 'TX_APPORT_SE_BRP',
 'LTV_OCTROI_BRP',
 'LTV_ACPR_BRP',
 'TOP_SURFINANCEMENT_BRP',
 'top_exist_conso_revo_BRP',
 'NBR_DUREE_TOT_PRET_CRI',
 'MNT_PRET_CRI',
 'PCT_NEGOC_CRI',
 'PCT_TEG_TAEG_CRI',
 'MNT_TOT_ASSURANCE_CRI',
 'MNT_COUT_TOT_CREDIT_CRI',
 'COUT_ACQ_BRP',
 'COUT_DOSSIER_BRP',
 